In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as functions
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader#, IterableDataset

In [3]:
# Load the dataset using Pandas
data = pd.read_csv('diabetes.csv')
print(data.shape)
data.head()

(768, 8)


,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [4]:
# For x: Extract out the dataset from all the rows (all samples) and all columns except last column (all features). 
# For y: Extract out the last column (which is the label)
# Convert both to numpy using the .values method
x = data.iloc[:,0:-1].values
print(x)
y_string= list(data.iloc[:,-1])
print(type(y_string))

[[  6.  148.   72.  ...   0.   33.6  50. ]
 [  1.   85.   66.  ...   0.   26.6  31. ]
 [  8.  183.   64.  ...   0.   23.3  32. ]
 ...
 [  5.  121.   72.  ... 112.   26.2  30. ]
 [  1.  126.   60.  ...   0.   30.1  47. ]
 [  1.   93.   70.  ...   0.   30.4  23. ]]
<class 'list'>


In [5]:
# Lets have a look some samples from our data
print(x[:3]) #same as dataframe.head(3)
print(y_string[:3])

[[  6.  148.   72.   35.    0.   33.6  50. ]
 [  1.   85.   66.   29.    0.   26.6  31. ]
 [  8.  183.   64.    0.    0.   23.3  32. ]]
['positive', 'negative', 'positive']


In [6]:
# Our neural network only understand numbers! So convert the string to labels
y_int = []
for string in y_string:
    if string == 'positive':
        y_int.append(1)
    else:
        y_int.append(0)

In [7]:
# Now convert to an array
y = np.array(y_int, dtype = 'float64')

In [8]:
# Feature Normalization. All features should have the same range of values (-1,1)
sc = StandardScaler()
x = sc.fit_transform(x)

In [9]:
# Now we convert the arrays to PyTorch tensors
x = torch.tensor(x)
# We add an extra dimension to convert this array to 2D
y = torch.tensor(y).unsqueeze(1)

In [10]:
print(x.shape)
print(y.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [11]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [12]:
dataset = Dataset(x,y)

In [13]:
type(dataset)

__main__.Dataset

In [14]:
len(dataset)

768

In [15]:
# Load the data to your dataloader for batch processing and shuffling
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)
print(type(train_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [16]:
# Let's have a look at the data loader
print("There is {} batches in the dataset".format(len(train_loader)))
for (x,y) in train_loader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break

There is 24 batches in the dataset
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


In [37]:
# Building a Class for the model neural network
class DiabetesSimpleNeuralNetModel(nn.Module):
    def __init__(self, input_features):
        super(DiabetesSimpleNeuralNetModel, self).__init__()
        self.inputlayer = nn.Linear(input_features, 5) #can be passed as parameters as well 
        self.hiddenlayer = nn.Linear(5, 3)  #can be passed as parameters as well
        self.outputlayer = nn.Linear(3, 1)  #can be passed as parameters as well
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.relu = functions.relu
        #######Variations: Call to initialize weights:
        self.init_weights()

    ##########Variations: Initializing weights manually:        
    def init_weights(self):
        nn.init.kaiming_normal_(self.inputlayer.weight)
        nn.init.uniform_(self.hiddenlayer.weight, a = 0.0, b = 3)
        nn.init.normal_(self.outputlayer.weight, mean = 0.0, std = 1.0)
        # nn.init.xavier_uniform_(self.outputlayer.weight, gain = 1.0)
        #nn.init.xavier_normal_(self.outputlayer.weight, gain = 1.0)
        nn.init.constant_(self.inputlayer.bias, 0.4) #weights can also be done
        
    def forward(self, x):
        inputlayerout = self.inputlayer(x)
        out1fn = self.relu(inputlayerout)     
        ####Variations: out1fn = self.tanh()
        hiddenlayerout = self.hiddenlayer(out1fn)
        out2fn = self.relu(hiddenlayerout)    # can be self.tanh or any other activation functions
        finaloutput = self.outputlayer(out2fn)
        sigmoidfinaloutput = self.sigmoid(finaloutput)
        return sigmoidfinaloutput

In [20]:
# Create the network (an object of the Net class)
net = DiabetesSimpleNeuralNetModel(x.shape[1])
net

DiabetesSimpleNeuralNetModel(
  (inputlayer): Linear(in_features=7, out_features=5, bias=True)
  (hiddenlayer): Linear(in_features=5, out_features=3, bias=True)
  (outputlayer): Linear(in_features=3, out_features=1, bias=True)
  (tanh): Tanh()
  (sigmoid): Sigmoid()
)

In [21]:
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
criterion = torch.nn.BCELoss(size_average=True) 
####Variations:criterion = torch.nn.CrossEntropyLoss()

C:\Anaconda\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [22]:
# We will use SGD with momentum with a learning rate of 0.1
optimizer = torch.optim.SGD(net.parameters(), lr=0.1) #, momentum=0.9
####Variations: optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

In [29]:
# Train the network 
num_epochs = 20
running_loss = 0
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = net(inputs) #is same as net.forward(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (we don't want to accumulate gradients)
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient
        optimizer.step()
        
    #Accuracy
    # Since we are using a sigmoid, we will need to perform some manual thresholding
    output = (output>0.5).float()
    ######Variations in case of multiple: _, outputs: output = torch.max(outputs.data, 1)    
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

Epoch 1/20, Loss: 0.520, Accuracy: 0.750
Epoch 2/20, Loss: 0.603, Accuracy: 0.719
Epoch 3/20, Loss: 0.496, Accuracy: 0.750
Epoch 4/20, Loss: 0.562, Accuracy: 0.719
Epoch 5/20, Loss: 0.460, Accuracy: 0.844
Epoch 6/20, Loss: 0.477, Accuracy: 0.781
Epoch 7/20, Loss: 0.413, Accuracy: 0.844
Epoch 8/20, Loss: 0.388, Accuracy: 0.844
Epoch 9/20, Loss: 0.679, Accuracy: 0.688
Epoch 10/20, Loss: 0.355, Accuracy: 0.844
Epoch 11/20, Loss: 0.517, Accuracy: 0.750
Epoch 12/20, Loss: 0.389, Accuracy: 0.875
Epoch 13/20, Loss: 0.547, Accuracy: 0.750
Epoch 14/20, Loss: 0.406, Accuracy: 0.875
Epoch 15/20, Loss: 0.367, Accuracy: 0.844
Epoch 16/20, Loss: 0.492, Accuracy: 0.781
Epoch 17/20, Loss: 0.551, Accuracy: 0.688
Epoch 18/20, Loss: 0.511, Accuracy: 0.812
Epoch 19/20, Loss: 0.351, Accuracy: 0.875
Epoch 20/20, Loss: 0.503, Accuracy: 0.750


In [34]:
#####Variations: Getting created Neural Network details 
print(net.parameters)
print('\n\n\n')
print(net.parameters())
print('\n\n\n')
print(list(net.parameters()))

<bound method Module.parameters of DiabetesSimpleNeuralNetModel(
  (inputlayer): Linear(in_features=7, out_features=5, bias=True)
  (hiddenlayer): Linear(in_features=5, out_features=3, bias=True)
  (outputlayer): Linear(in_features=3, out_features=1, bias=True)
  (tanh): Tanh()
  (sigmoid): Sigmoid()
)>




<generator object Module.parameters at 0x000001F18491D5C8>




[Parameter containing:
tensor([[ 0.2389,  0.6589, -0.1649,  0.3268, -0.2096,  0.1507,  0.1267],
        [-0.2613,  0.0737, -1.0609,  0.4494,  0.0722,  0.6821, -0.5706],
        [-0.4947,  0.9146,  0.4219,  0.0639, -0.7132,  0.9676, -0.5428],
        [-0.1781, -1.3371, -0.1908,  0.7297, -0.5981, -0.4271, -0.4646],
        [ 0.3609,  0.5077, -0.2068, -0.5160,  0.3741,  0.6768, -0.1496]],
       requires_grad=True), Parameter containing:
tensor([ 0.9529, -0.2378,  0.2045,  0.2075,  0.2865], requires_grad=True), Parameter containing:
tensor([[-0.3122,  1.2898, -0.0802, -1.4495, -0.6280],
        [-0.8942,  0.2435, -0.8998, -

In [36]:
#####Variations: Getting created Neural Network Layer level details 
print(net.inputlayer.weight)
print('\n')
print(net.hiddenlayer.weight)
print('\n')
print(net.outputlayer.weight)

Parameter containing:
tensor([[ 0.2389,  0.6589, -0.1649,  0.3268, -0.2096,  0.1507,  0.1267],
        [-0.2613,  0.0737, -1.0609,  0.4494,  0.0722,  0.6821, -0.5706],
        [-0.4947,  0.9146,  0.4219,  0.0639, -0.7132,  0.9676, -0.5428],
        [-0.1781, -1.3371, -0.1908,  0.7297, -0.5981, -0.4271, -0.4646],
        [ 0.3609,  0.5077, -0.2068, -0.5160,  0.3741,  0.6768, -0.1496]],
       requires_grad=True)


Parameter containing:
tensor([[-0.3122,  1.2898, -0.0802, -1.4495, -0.6280],
        [-0.8942,  0.2435, -0.8998, -0.3440, -0.1880],
        [ 1.3784, -0.7096,  0.2665, -0.1647,  0.7501]], requires_grad=True)


Parameter containing:
tensor([[1.0269, 0.3202, 0.7876]], requires_grad=True)
